# Holder Wallet Conformance Flows (v3) - CTWalletCrossDeferred

# 0.0 Initial setup

## 0.1 Setup conformance

In [19]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [20]:
from ebsi_wallet.did_key import KeyDid, PublicKeyJWK

key_did = KeyDid()

# generate keypair
key_did.create_keypair()

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    y=key_did.public_key_jwk['y']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

Decentralised identifier:  did:key:z2dmzD81cgPx8Vki7JbuuMmFYrWPgYoytykUZ3eyqht1j9Kbp2sTAbTnXhViu5VQuwi35oFPWrVcsWLbdVYi3Hy8KpXFLTsKKkmxfXHkKkUos8KQsochfXAGFdh3aasgtrycmPa7mXwCH9xsEiWyuPNnwiBJCtnDkYETgu1uE9cXb83o9v


## 1.1 Initiate Credential Issuance

In [21]:
from ebsi_wallet.util import parse_query_string_parameters_from_url
from ebsi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

credential_type =  CredentialTypes.CTWalletCrossDeferred
qr_code_data = await fetch_credential_offer(key_did._did, credential_type)
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_server)
console.log("Authorization server configuration: ", auth_server_configuration)


[18:08:11] Credential offer URI:                                                                   ]8;id=294042;file:///tmp/ipykernel_41162/3425914532.py\3425914532.py]8;;\:]8;id=469746;file:///tmp/ipykernel_41162/3425914532.py#15\15]8;;\
           https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/offers/43f26103-e4cf-445d-97                 
           2b-2ab629faaeea                                                                                         

[18:08:14] Credential offer:                                                                       ]8;id=595788;file:///tmp/ipykernel_41162/3425914532.py\3425914532.py]8;;\:]8;id=462613;file:///tmp/ipykernel_41162/3425914532.py#18\18]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               credentials=[                                                                                       
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'CTWalletCrossDeferred'                                                                 
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       }                                                                                           
                   }                                                                                               
               ],                                                                                                  
               grants={                                                                                            
                   'authorization_code': {                                                                         
                       'issuer_state':                                                                             
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNF                 
           ZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1                 
           BnWW95dHlrVVozZXlxaHQxajlLYnAyc1RBYlRuWGhWaXU1VlF1d2kzNW9GUFdyVmNzV0xiZFZZaTNIeThLcFhGT                 
           FRzS0trbXhmWEhrS2tVb3M4S1Fzb2NoZlhBR0ZkaDNhYXNndHJ5Y21QYTdtWHdDSDl4c0VpV3l1UE5ud2lCSkN0                 
           bkRrWUVUZ3UxdUU5Y1hiODNvOXYiLCJjcmVkZW50aWFsX3R5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiw                 
           iVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc0RlZmVycmVkIl0sImlhdCI6MTY4NzE3ODI5MC                 
           wiZXhwIjoxNjg3MTc4NTkwLCJpc3MiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hb                 
           mNlL3YzL2lzc3Vlci1tb2NrIiwiYXVkIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5ldS9jb25mb3Jt                 
           YW5jZS92My9hdXRoLW1vY2siLCJzdWIiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW9                 
           5dHlrVVozZXlxaHQxajlLYnAyc1RBYlRuWGhWaXU1VlF1d2kzNW9GUFdyVmNzV0xiZFZZaTNIeThLcFhGTFRzS0                 
           trbXhmWEhrS2tVb3M4S1Fzb2NoZlhBR0ZkaDNhYXNndHJ5Y21QYTdtWHdDSDl4c0VpV3l1UE5ud2lCSkN0bkRrW                 
           UVUZ3UxdUU5Y1hiODNvOXYifQ.s6qFlZnzICHzR2uWGZW4myVx9bXkpBQgRHoO20S0tEPdZYnc_RG7cB_60y9iR                 
           Mzdk1CkD6HLkGdPTKQ4kz5hxg'    

           Credential issuer configuration:                                                        ]8;id=454994;file:///tmp/ipykernel_41162/3425914532.py\3425914532.py]8;;\:]8;id=382603;file:///tmp/ipykernel_41162/3425914532.py#21\21]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               authorization_server='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                    
               credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/cre                 
           dential',                                                                                               
               deferred_credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer                 
           -mock/credential_deferred',                                                                             
               credentials_supported=[                                                                             
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAuthorisationToOnboard'                                                      
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       },                                                                                          
                       'display': [                                                                                
                           {'name': 'Verifiable Authorisation to onboard', 'locale': 'en-GB'}                      
                       ]                                                                                           
                   },                                                                                              
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAccreditation',                                                              
                           'VerifiableAccreditationToAttest'                                                       
                       ],                                                                                          
                       'trust_framework':

[18:08:15] Authorization server configuration:                                                     ]8;id=404234;file:///tmp/ipykernel_41162/3425914532.py\3425914532.py]8;;\:]8;id=502223;file:///tmp/ipykernel_41162/3425914532.py#24\24]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=[                                                                                     
                   'https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                          
               ],                                                                                                  
               issuer='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                                  
               authorization_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/au                 
           thorize',                                                                                               
               token_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/token',                    
               jwks_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/jwks',                           
               scopes_supported=['openid'],                                                                        
               response_types_supported=['vp_token', 'id_token'],                                                  
               response_modes_supported=['query'],                                                                 
               grant_types_supported=['authorization_code'],                                                       
               subject_types_supported=['public'],                                                                 
               id_token_signing_alg_values_supported=['ES256'],                                                    
               request_object_signing_alg_values_supported=['ES256'],                                              
               request_parameter_supported=True,                                                                   
               request_uri_parameter_supported=True,                                                               
               token_endpoint_auth_methods_supported=['private_key_jwt'],                                          
               request_authentication_methods_supported={                                                          
                   'authorization_endpoint': ['request_object']                                                    
               },                                                                                                  
               vp_formats_supported={                                                                              
                   'jwt_vp': {'alg_values_supported': ['ES256']},                                                  
                   'jwt_vc': {'alg_values_supported': ['ES256']}                                                   
               },                                                                                                  
               subject_syntax_types_supported=['did:key', 'did:ebsi'],                                             
               subject_syntax_types_discriminations=['did:key:jwk_jcs-pub', 'did:ebsi:v1'],                        
               subject_trust_frameworks_supported=['ebsi'],                                                        
               id_token_types_supported=['subject_signed_id_token', 'attester_signed_id_token']                    
           )                                                                                                       

## 1.2 Perform authorization request and obtain ID token request

In [22]:
from ebsi_wallet.siop_auth.util import (
    perform_authorization, 
    AuthorizationRequestQueryParams,
    get_authorization_response_query_params,
    generate_code_challenge,
    generate_code_verifier
)
import uuid
import json

state = str(uuid.uuid4())
nonce = str(uuid.uuid4())
code_verifier = generate_code_verifier()

authorization_details = [{"type":"openid_credential","format":"jwt_vc","types":credential_offer.credentials[0].get("types"),"locations": ["https://api-conformance.ebsi.eu/conformance/v3/issuer-mock"]}]
redirect_uri = "http://localhost:8080"
client_metadata = {"vp_formats_supported":{"jwt_vp":{"alg":["ES256"]},"jwt_vc":{"alg":["ES256"]}},"response_types_supported":["vp_token","id_token"],"authorization_endpoint":redirect_uri}
authorization_request_query_params = AuthorizationRequestQueryParams(
    response_type="code",
    scope="openid",
    state=state,
    client_id=key_did._did,
    authorization_details=json.dumps(authorization_details, separators=(',', ':')),
    redirect_uri=redirect_uri,
    nonce=nonce,
    code_challenge=generate_code_challenge(code_verifier),
    code_challenge_method="S256",
    client_metadata=json.dumps(client_metadata, separators=(',', ':')),
    issuer_state=credential_offer.grants.get("authorization_code").get("issuer_state")
)

auth_resp = await perform_authorization(auth_server_configuration.authorization_endpoint, authorization_request_query_params)
auth_resp_uri = str(auth_resp).split("Location': '")[1].split("'")[0]
console.log("Authorization response URI: ", auth_resp_uri)
auth_resp_query_params = get_authorization_response_query_params(auth_resp_uri)
console.log("Authorization response query params: ", auth_resp_query_params)

[18:08:16] Authorization response URI:                                                             ]8;id=773378;file:///tmp/ipykernel_41162/1836098038.py\1836098038.py]8;;\:]8;id=496786;file:///tmp/ipykernel_41162/1836098038.py#34\34]8;;\
           http://localhost:8080?state=38c1b779-cd3e-47ae-b704-356d0340b324&client_id=https%3A%2F%                 
           2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock&redirect_uri=https%3A%2F%2Fapi                 
           -conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock%2Fdirect_post&response_type=id_toke                 
           n&response_mode=direct_post&scope=openid&nonce=c690b792-2a48-4df6-a9ee-3836e6c6d371&req                 
           uest_uri=https%3A%2F%2Fapi-conformance.ebsi.eu%2Fconformance%2Fv3%2Fauth-mock%2Frequest                 
           _uri%2F5601eb2f-0655-4fdd-8aba-ad26d543705d                                                             

           Authorization response query params:                                                    ]8;id=790217;file:///tmp/ipykernel_41162/1836098038.py\1836098038.py]8;;\:]8;id=941709;file:///tmp/ipykernel_41162/1836098038.py#36\36]8;;\
           AuthorizationResponseQueryParams(                                                                       
               state='38c1b779-cd3e-47ae-b704-356d0340b324',                                                       
               client_id='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                               
               redirect_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                 
           ,                                                                                                       
               response_type='id_token',                                                                           
               response_mode='direct_post',                                                                        
               scope='openid',                                                                                     
               nonce='c690b792-2a48-4df6-a9ee-3836e6c6d371',                                                       
               request_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/request_uri/5                 
           601eb2f-0655-4fdd-8aba-ad26d543705d'                                                                    
           )                                                                                                       

## 1.3 Send ID token

In [23]:
from ebsi_wallet.siop_auth.util import send_id_token_response

id_token = key_did.generate_id_token(auth_server_uri=auth_resp_query_params.client_id, nonce=auth_resp_query_params.nonce)
auth_code_response = await send_id_token_response(auth_resp_query_params.redirect_uri, id_token, auth_resp_query_params.state)
auth_code_response = str(auth_code_response).split("Location': '")[1].split("'")[0]
state = parse_query_string_parameters_from_url(auth_code_response).get("state")[0]
auth_code = parse_query_string_parameters_from_url(auth_code_response).get("code")[0]
console.log("Authorization code: ", auth_code)

[18:08:17] Authorization code:  bae88117-93e0-461c-8138-e639f1b1e82e                                ]8;id=15944;file:///tmp/ipykernel_41162/1546471797.py\1546471797.py]8;;\:]8;id=147731;file:///tmp/ipykernel_41162/1546471797.py#8\8]8;;\

## 1.4 Exchange code for access token

In [24]:
from ebsi_wallet.siop_auth.util import exchange_auth_code_for_access_token

token_uri = auth_resp_query_params.client_id + "/token"
access_token_response = await exchange_auth_code_for_access_token(token_uri, key_did._did, auth_code, code_verifier)
console.log("Access token response: ", access_token_response)

[18:08:18] Access token response:                                                                    ]8;id=319455;file:///tmp/ipykernel_41162/474231906.py\474231906.py]8;;\:]8;id=314556;file:///tmp/ipykernel_41162/474231906.py#5\5]8;;\
           AccessTokenResponse(                                                                                    
               access_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWV               
           ZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjU4MjRmYzlhLTY2ODItNGEzMi05ZWNkLTRjZjA4Y               
           2ZhOTIxZCIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudG               
           lhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV               
           1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVy               
           aWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRDcm9zc0RlZmVycmVkIl19XSwiY19ub25jZSI6ImFkMWY0MTMyL               
           WExYjktNDdjZi04YjhhLWI0MWU2MzhmNmExZCIsImNfbm9uY2VfZXhwaXJlc19pbiI6ODY0MDAsImNsaWVudF9pZC               
           I6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticDJzVEFiVG5               
           YaFZpdTVWUXV3aTM1b0ZQV3JWY3NXTGJkVllpM0h5OEtwWEZMVHNLS2tteGZYSGtLa1VvczhLUXNvY2hmWEFHRmRo               
           M2Fhc2d0cnljbVBhN21Yd0NIOXhzRWlXeXVQTm53aUJKQ3RuRGtZRVRndTF1RTljWGI4M285diJ9LCJpc3MiOiJod               
           HRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2F1dGgtbW9jayIsImF1ZCI6WyJodH               
           RwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInN1YiI6ImR               
           pZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticDJzVEFiVG5YaFZp               
           dTVWUXV3aTM1b0ZQV3JWY3NXTGJkVllpM0h5OEtwWEZMVHNLS2tteGZYSGtLa1VvczhLUXNvY2hmWEFHRmRoM2Fhc               
           2d0cnljbVBhN21Yd0NIOXhzRWlXeXVQTm53aUJKQ3RuRGtZRVRndTF1RTljWGI4M285diIsImlhdCI6MTY4NzE3OD               
           I5NywiZXhwIjoxNjg3MjY0Njk3fQ.gIWUKdAWrnYk95Ys5fCG8RoP2FoMIv3fughjhTq9hjTHheJDjyc5GyZpRjSq               
           TfKRrXqC0PKqzgaaAHCST6BKlg',                                                                            
               token_type='Bearer',                                                                                
               expires_in=86400,                                                                                   
               id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUWl               
           GbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6ImYxYjJlNzlmLThlZDEtNGQzNS1hNTI4LTdiZWMxNmI1Y               
           2JjZCIsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2               
           NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2J               
           wMnNUQWJUblhoVml1NVZRdXdpMzVvRlBXclZjc1dMYmRWWWkzSHk4S3BYRkxUc0tLa214ZlhIa0trVW9zOEtRc29j               
           aGZYQUdGZGgzYWFzZ3RyeWNtUGE3bVh3Q0g5eHNFaVd5dVBObndpQkpDdG5Ea1lFVGd1MXVFOWNYYjgzbzl2IiwiY               
           XVkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwMnNUQW               
           JUblhoVml1NVZRdXdpMzVvRlBXclZjc1dMYmRWWWkzSHk4S3BYRkxUc0tLa214ZlhIa0trVW9zOEtRc29jaGZYQUd               
           GZGgzYWFzZ3RyeWNtUGE3bVh3Q0g5eHNFaVd5dVBObndpQkpDdG5Ea1lFVGd1MXVFOWNYYjgzbzl2IiwiaWF0Ijox               
           Njg3MTc4Mjk3LCJleHAiOjE2ODcxNzg0MTd9.nrBd7-OZjWd3MjbInG0kKPP8q1WBSjJoH51coHCvJsU3czgAF3th               
           Y03Plp8u2udk6x8yMjfh_jwAHU5dxHJ3tw',                                                                    
               c_nonce='ad1f4132-a1b9-47cf-8b8a-b41e638f6a1d',                                                     
               c_nonce_expires_in=86400     

## 1.5 Request credential to get acceptance token to initiate deferred flow 

In [25]:
from ebsi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_request(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
acceptance_token_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_offer.credentials[0].get("types"))
console.log("Acceptance token response: ", acceptance_token_response)## 1.6 Request credential using deferred endpoint

           Credential request JWT:                                                                   ]8;id=925238;file:///tmp/ipykernel_41162/380321948.py\380321948.py]8;;\:]8;id=230902;file:///tmp/ipykernel_41162/380321948.py#4\4]8;;\
           eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVW               
           jNleXFodDFqOUticDJzVEFiVG5YaFZpdTVWUXV3aTM1b0ZQV3JWY3NXTGJkVllpM0h5OEtwWEZMVHNLS2tteGZYSG               
           tLa1VvczhLUXNvY2hmWEFHRmRoM2Fhc2d0cnljbVBhN21Yd0NIOXhzRWlXeXVQTm53aUJKQ3RuRGtZRVRndTF1RTl               
           jWGI4M285diN6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JwMnNUQWJUblho               
           Vml1NVZRdXdpMzVvRlBXclZjc1dMYmRWWWkzSHk4S3BYRkxUc0tLa214ZlhIa0trVW9zOEtRc29jaGZYQUdGZGgzY               
           WFzZ3RyeWNtUGE3bVh3Q0g5eHNFaVd5dVBObndpQkpDdG5Ea1lFVGd1MXVFOWNYYjgzbzl2IiwidHlwIjoib3Blbm               
           lkNHZjaS1wcm9vZitqd3QifQ.eyJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1h               
           bmNlL3YzL2lzc3Vlci1tb2NrIiwiZXhwIjoxNjg3MjY0Njk4LCJpYXQiOjE2ODcxNzgyOTgsImlzcyI6ImRpZDprZ               
           Xk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticDJzVEFiVG5YaFZpdTVWUX               
           V3aTM1b0ZQV3JWY3NXTGJkVllpM0h5OEtwWEZMVHNLS2tteGZYSGtLa1VvczhLUXNvY2hmWEFHRmRoM2Fhc2d0cnl               
           jbVBhN21Yd0NIOXhzRWlXeXVQTm53aUJKQ3RuRGtZRVRndTF1RTljWGI4M285diIsIm5vbmNlIjoiYWQxZjQxMzIt               
           YTFiOS00N2NmLThiOGEtYjQxZTYzOGY2YTFkIn0.hSUFlbiBE-p7M58JxHk8azMFm3OdQOv9BKFOfHCeanslPaLgq               
           A25LxHoBNF_IM-CmIoFNO68_mxVPcIeX17neA                                                                   

           Acceptance token response:                                                                ]8;id=121366;file:///tmp/ipykernel_41162/380321948.py\380321948.py]8;;\:]8;id=700251;file:///tmp/ipykernel_41162/380321948.py#6\6]8;;\
           {'acceptance_token': '-AUUc5cL05lumbg1rmHbg51eZGD4e5GMxecEPUhngJE'}                                     

## 1.6 Request credential using deferred endpoint

In [27]:
from ebsi_wallet.siop_auth.util import send_deferred_credential_request
from time import sleep

console.log("Waiting for 10 seconds...")
sleep(10)
console.log("Sending deferred credential request...")
credential_response = await send_deferred_credential_request(credential_issuer_configuration.deferred_credential_endpoint, acceptance_token_response.get("acceptance_token"))
console.log("Credential response: ", credential_response)

[18:09:21] Waiting for 10 seconds...                                                                ]8;id=479448;file:///tmp/ipykernel_41162/1425646747.py\1425646747.py]8;;\:]8;id=61873;file:///tmp/ipykernel_41162/1425646747.py#4\4]8;;\

[18:09:31] Sending deferred credential request...                                                   ]8;id=993613;file:///tmp/ipykernel_41162/1425646747.py\1425646747.py]8;;\:]8;id=249400;file:///tmp/ipykernel_41162/1425646747.py#6\6]8;;\

[18:09:32] Credential response:                                                                     ]8;id=778033;file:///tmp/ipykernel_41162/1425646747.py\1425646747.py]8;;\:]8;id=594952;file:///tmp/ipykernel_41162/1425646747.py#8\8]8;;\
           {                                                                                                       
               'title': 'Internal Server Error',                                                                   
               'status': 500,                                                                                      
               'type': 'about:blank',                                                                              
               'detail': 'The server encountered an internal error and was unable to complete your                 
           request'                                                                                                
           }                                                                                                       